<a href="https://colab.research.google.com/github/plnu-biomechanics/kin6015/blob/main/notebooks/kin6015_lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://www.pointloma.edu/sites/default/files/styles/basic_page/public/images/PLNU_Biomechanics_Lab_green_yellowSD_HiRes.png" width=400>

## **KIN 6015 Biomechanical Basis of Human Movement**
Instructor: Arnel Aguinaldo, PhD

**Lab 1 Data Processing**

In this lab, gait analysis data was collected with the marker-based and markerless motion capture systems and spatiotemporal metrics and inverse kinematics (IK) were estimated using Visual3D. The data were then exported as text (*.txt) files and uploaded to the class repository in the lab's [GitHub](https://github.com/plnu-biomechanics).

To further process the data for this lab, follow the steps in this **Colab notebook**, which contains instructions and sample code on how to wrangle and analyze the data.


### Create your own Colab Notebook

1. Go to **File -> New notebook in Drive** to open a new notebook in your Python environment:<br>
<img src="https://raw.githubusercontent.com/plnu-biomechanics/kin6015/main/notebooks/images/file_notebook.png" width=450>

2. Rename your Colab notebook using this naming format: **lastname_group_lab#.ipynb** (e.g., "aguinaldo_targaryen_lab1.ipynb")
3. Click on the **+ Code** option above to insert a new code cell: <br>
<img src="https://raw.githubusercontent.com/plnu-biomechanics/kin6015/main/notebooks/images/addcode.png" width=280>

4. The data you will parse and analyze for this lab will be copied from the lab's GitHub and temporarily stored in your Colab working directory, which can be accessed by clicking on the folder icon in the left menu:<br>
<img src="https://raw.githubusercontent.com/plnu-biomechanics/kin6015/main/notebooks/images/colab_folder.png" width=400>

5. Copy the following lines of code to import the packages needed for this analysis and to load the data files into your working directory. Be sure to update the `GROUP` variable with your group's name. **Note**: These files are "runtime" access only, meaning they are only temporarily stored in your working directory and show up when your notebook is in session. However, the following code cell allows you to clone the zipped files to the working directory each time it is executed.


In [1]:
import urllib.request
import zipfile
import os

# --------------------------------------------------
# STUDENT INPUT (edit only this line; case-sensitive)
# --------------------------------------------------
GROUP = "targaryen"   # e.g., "targaryen", "stark", "lannister", "martel"

# --------------------------------------------------
# Configuration (do NOT edit below)
# These lines create a directory for this lab in your
# Colab working directory.
# --------------------------------------------------
zip_dir = "kin6015/lab1"
os.makedirs(zip_dir, exist_ok=True)

zip_filename = f"spring2026_lab1_{GROUP}.zip"

url = (
    "https://raw.githubusercontent.com/"
    "plnu-biomechanics/kin6015/main/"
    f"labs/{zip_filename}"
)

zip_path = os.path.join(zip_dir, zip_filename)

# --------------------------------------------------
# Download zip file
# --------------------------------------------------
urllib.request.urlretrieve(url, zip_path)

# --------------------------------------------------
# Extract contents from the zipped file
# --------------------------------------------------
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(zip_dir)

print("Extracted files in lab directory:")
print(os.listdir(zip_dir))


Extracted files in lab directory:
['Targaryen_MB_05.txt', 'Targaryen_ML_03.txt', 'Targaryen_ML_01.txt', 'Targaryen_ML_05.txt', 'Targaryen_MB_03.txt', 'spring2026_lab1_targaryen.zip', 'Targaryen_MB_01.txt', 'Targaryen_ML_04.txt', 'Targaryen_MB_04.txt', 'Targaryen_MB_02.txt', 'Targaryen_ML_02.txt']


# Gemini Task
#### Prompt:
Create a single pandas DataFrame named `combined_df` by iterating through all `.txt` files in the `kin6015/lab1` directory, applying the `parse_txt_file` function to each, and then concatenating the results. Finally, display the head of `combined_df` to verify its structure and content.

## Parse All Files and Combine Data

### Subtask:
Iterate through all '.txt' files located in the 'kin6015/lab1' directory. Apply the updated `parse_txt_file` function to each file. Collect all the resulting DataFrames into a list, and then concatenate them into a single master DataFrame named `combined_df`. This step will re-attempt to process all files with the corrected parsing logic.


In [11]:
import pandas as pd
import os

def parse_txt_file(filepath):
  """Reads a text file, extracts variable names and time-series data, identifies the condition,
  and returns a pandas DataFrame.

  Args:
    filepath (str): The full path to the text file.

  Returns:
    pd.DataFrame: A DataFrame containing the extracted data, condition, and filename.
  """
  with open(filepath, 'r') as file:
    lines = file.readlines()

  # Check if file has enough lines for the expected structure
  if len(lines) < 6: # Need at least 6 lines (5 header, 1 data)
      raise ValueError(f"File {os.path.basename(filepath)} has too few lines to parse correctly.")

  # Extract variable names from the 2nd line (index 1), using tab as delimiter
  variable_names = lines[1].strip().split('\t')

  # Read time-series data starting from the 6th line (index 5), using tab as delimiter
  data_lines = [line.strip().split('\t') for line in lines[5:] if line.strip()]

  # Ensure data_lines is not empty before checking lengths
  if not data_lines:
      raise ValueError(f"No data found in file {os.path.basename(filepath)} after header.")

  # Dynamically add 'Frame' if data rows have one more column than variable names
  if len(data_lines[0]) == len(variable_names) + 1:
      variable_names.insert(0, 'Frame')

  # Ensure variable names and data lines match in length AFTER potential adjustment
  if len(variable_names) != len(data_lines[0]):
      raise ValueError(f"Column name count ({len(variable_names)}) does not match data column count ({len(data_lines[0])}) in file {os.path.basename(filepath)}.")

  # Create DataFrame
  df = pd.DataFrame(data_lines, columns=variable_names)

  # Convert numeric columns to appropriate data types
  for col in df.columns:
    try:
      df[col] = pd.to_numeric(df[col])
    except ValueError:
      pass # Keep as string if not numeric

  # Determine 'condition' from filename
  filename = os.path.basename(filepath)
  if 'MB' in filename.upper(): # Changed to upper() for accurate matching of 'MB'
    condition = 'marker-based'
  elif 'ML' in filename.upper(): # Changed to upper() for accurate matching of 'ML'
    condition = 'markerless'
  else:
    condition = 'unknown'

  # Add 'condition' column
  df['condition'] = condition

  # Add 'filename' column
  df['filename'] = filename

  return df

print("Updated function 'parse_txt_file' to handle tab-delimited data, correct line indexing, and dynamic 'Frame' column and case-sensitive condition check.")

Updated function 'parse_txt_file' to handle tab-delimited data, correct line indexing, and dynamic 'Frame' column and case-sensitive condition check.


**Reasoning**:
The `parse_txt_file` function has been updated to handle the 'Frame' column and tab-delimited data. Now, I need to re-run the code to iterate through all `.txt` files in the `kin6015/lab1` directory, apply the updated `parse_txt_file` function to each, collect the resulting DataFrames, and then concatenate them into a single `combined_df`. Finally, the following code displays the head of `combined_df` to verify the structure and content, as per the subtask instructions.



In [13]:
import re
import os

all_dataframes = []

# Get a list of all files in the directory. zip_dir is defined in a previous cell.
file_list = os.listdir(zip_dir)

# Filter for .txt files and exclude the zip file
text_files = [f for f in file_list if f.endswith('.txt') and not f.endswith('.zip')]

for filename in text_files:
    filepath = os.path.join(zip_dir, filename)
    try:
        df = parse_txt_file(filepath)
        all_dataframes.append(df)
    except Exception as e:
        print(f"Error processing file {filename}: {e}")

# Concatenate all DataFrames into a single DataFrame
if all_dataframes:
    combined_df = pd.concat(all_dataframes, ignore_index=True)
    # Let's clean up the variable names by removing
    # '_MEAN' or '_Mean' from column names
    combined_df.columns = [re.sub(r'_MEAN', '', col) for col in combined_df.columns]
    combined_df.columns = [re.sub(r'_Mean', '', col) for col in combined_df.columns]

    if 'Right Ankle Angles' in combined_df.columns:
      combined_df = combined_df.rename(columns={'Right Ankle Angles': 'Right Ankle Dorsiflexion'})
      print("Column 'Right Ankle Angles' renamed to 'Right Ankle Dorsiflexion'.")
    else:
      print("Column 'Right Ankle Angles' not found in combined_df.")

    # List to hold individual Series that represent dorsiflexion data
    dorsiflexion_data_series = []

    # Define all potential names for the dorsiflexion column
    potential_dorsiflexion_cols = [
        'Right Ankle Dorsiflexion',
        'Right Ankle Angles'
    ]

    # Iterate through all columns in the combined_df by index and name
    # This correctly handles cases where column names are duplicated by accessing via iloc
    for idx, col_name_in_df in enumerate(combined_df.columns):
        if col_name_in_df in potential_dorsiflexion_cols:
            # Extract the Series using iloc to get a single column Series
            dorsiflexion_data_series.append(combined_df.iloc[:, idx])

    # Initialize the consolidated Series as an empty Series with the correct index
    consolidated_dorsiflexion = pd.Series(index=combined_df.index, dtype=float)

    # Merge data from all identified dorsiflexion series using fillna
    for s in dorsiflexion_data_series:
        consolidated_dorsiflexion = consolidated_dorsiflexion.fillna(s)

    # Now, we need to ensure combined_df only has *one* 'Right Ankle Dorsiflexion' column.
    # Reconstruct the DataFrame to drop all previous versions of the dorsiflexion related
    # columns and then add the new consolidated one.

    # Identify all columns that are NOT related to Right Ankle Dorsiflexion
    unique_non_dorsiflexion_cols = []
    for col_name in combined_df.columns:
        if col_name not in potential_dorsiflexion_cols:
            if col_name not in unique_non_dorsiflexion_cols: # Ensure uniqueness even if other columns were duplicated
                unique_non_dorsiflexion_cols.append(col_name)

    # Create a clean DataFrame from the unique non-dorsiflexion columns
    clean_df = combined_df[unique_non_dorsiflexion_cols].copy()

    # Add the single consolidated 'Right Ankle Dorsiflexion' Series
    clean_df['Right Ankle Dorsiflexion'] = consolidated_dorsiflexion

    # Update combined_df to be this new clean DataFrame
    combined_df = clean_df

    print("Displaying head of combined_df with updated column names:")
    display(combined_df.head())

else:
    combined_df = pd.DataFrame()
    print("No dataframes were parsed successfully to combine.")

Column 'Right Ankle Angles' renamed to 'Right Ankle Dorsiflexion'.
Displaying head of combined_df with updated column names:


,Frame,Right Knee Angles,Speed,Stride_Length,Right_Stance_Time,Ankle Plantarflexion at IC,Peak Right Knee Flexion,condition,filename,Right Ankle Dorsiflexion
0,1,3.54527,1.39355,0.99407,0.69667,-5.96584,67.92663,marker-based,Targaryen_MB_05.txt,-4.28667
1,2,5.48596,NaN,NaN,NaN,NaN,NaN,marker-based,Targaryen_MB_05.txt,-4.88219
2,3,6.19649,NaN,NaN,NaN,NaN,NaN,marker-based,Targaryen_MB_05.txt,-6.12455
3,4,6.24373,NaN,NaN,NaN,NaN,NaN,marker-based,Targaryen_MB_05.txt,-7.95542
4,5,6.29172,NaN,NaN,NaN,NaN,NaN,marker-based,Targaryen_MB_05.txt,-9.57410


In [16]:
current_columns = combined_df.columns.tolist()
new_columns = []
dorsiflexion_added = False

for col in current_columns:
    if col == 'Right Knee Angles':
        new_columns.append(col)
        # Add 'Right Ankle Dorsiflexion' immediately after 'Right Knee Angles'
        if 'Right Ankle Dorsiflexion' in current_columns and not dorsiflexion_added:
            new_columns.append('Right Ankle Dorsiflexion')
            dorsiflexion_added = True
    elif col != 'Right Ankle Dorsiflexion': # Avoid adding it again if already added or not the target column
        new_columns.append(col)

# Handle the case where 'Right Ankle Dorsiflexion' might be the very first column or not follow 'Right Knee Angles'
# This ensures all unique columns are present, and only one instance of 'Right Ankle Dorsiflexion'
final_new_columns = []
for col in new_columns:
    if col not in final_new_columns:
        final_new_columns.append(col)

# Ensure 'Right Ankle Dorsiflexion' is included if it was never added (e.g. if 'Right Knee Angles' wasn't present)
if 'Right Ankle Dorsiflexion' in current_columns and 'Right Ankle Dorsiflexion' not in final_new_columns:
    try:
        idx = final_new_columns.index('Right Knee Angles')
        final_new_columns.insert(idx + 1, 'Right Ankle Dorsiflexion')
    except ValueError:
        # If 'Right Knee Angles' is not found, just add it at the end for now or as appropriate.
        # Given the context, 'Right Knee Angles' is expected to exist.
        final_new_columns.append('Right Ankle Dorsiflexion')

# Remove any duplicates that might have been introduced by the logic (though it should be handled by 'final_new_columns' creation)
final_new_columns_unique = []
seen = set()
for item in final_new_columns:
    if item not in seen:
        final_new_columns_unique.append(item)
        seen.add(item)

# Reindex the DataFrame with the new column order
combined_df = combined_df[final_new_columns_unique]

print("Columns reordered successfully. Displaying head of combined_df with new column order:")
display(combined_df.head())

Columns reordered successfully. Displaying head of combined_df with new column order:


,Frame,Right Knee Angles,Right Ankle Dorsiflexion,Speed,Stride_Length,Right_Stance_Time,Ankle Plantarflexion at IC,Peak Right Knee Flexion,condition,filename
0,1,3.54527,-4.28667,1.39355,0.99407,0.69667,-5.96584,67.92663,marker-based,Targaryen_MB_05.txt
1,2,5.48596,-4.88219,NaN,NaN,NaN,NaN,NaN,marker-based,Targaryen_MB_05.txt
2,3,6.19649,-6.12455,NaN,NaN,NaN,NaN,NaN,marker-based,Targaryen_MB_05.txt
3,4,6.24373,-7.95542,NaN,NaN,NaN,NaN,NaN,marker-based,Targaryen_MB_05.txt
4,5,6.29172,-9.57410,NaN,NaN,NaN,NaN,NaN,marker-based,Targaryen_MB_05.txt


### Complete the Analysis

After you've completed this lab's analysis, do the following:

1. Copy and paste the above plots and tables to your lab report
2. Share your Colab notebook with me
3. Copy the link to your notebook and submit it along with your report for this lab on Canvas

## Install SPM1D Library

### Subtask:
Install the `spm1d` library, which is required for Statistical Parametric Mapping analysis. This is not a standard Colab library, so it needs to be installed first.


**Reasoning**:
The subtask requires installing the `spm1d` library, which can be done using `!pip install` in a code cell.



In [27]:
!pip install spm1d
print("spm1d library installed.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 57.0 MB/s eta 0:00:00
spm1d library installed.


# Task
To prepare `combined_df` for further analysis, reorder its columns to place 'Right Ankle Dorsiflexion' immediately after 'Right Knee Angles'. After reordering, display the first few rows of the DataFrame to confirm the new column arrangement.

## Reorder Columns

### Subtask:
Reorder columns in `combined_df` to place 'Right Ankle Dorsiflexion' immediately after 'Right Knee Angles'.


**Reasoning**:
To reorder the columns, I need to first get the current column order, then create a new list with the desired order, placing 'Right Ankle Dorsiflexion' right after 'Right Knee Angles'. After that, I will reindex the DataFrame using this new column list.

